In [1]:
/* Add Deps */
%AddDeps edu.stanford.nlp stanford-corenlp 3.7.0
%AddDeps com.google.protobuf protobuf-java 2.6.1
%AddDeps com.databricks spark-csv_2.10 1.5.0 --transitive

// Non-repo dependencies 
%AddJar file:lib/corenlp-models.jar
%AddJar file:SE/target/scala-2.10/se_2.10-1.1.jar

Marking edu.stanford.nlp:stanford-corenlp:3.7.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps6278997845129245926/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps6278997845129245926/https/repo1.maven.org/maven2/edu/stanford/nlp/stanford-corenlp/3.7.0/stanford-corenlp-3.7.0.jar
Marking com.google.protobuf:protobuf-java:2.6.1 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps6278997845129245926/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps6278997845129245926/https/repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
Marking com.databricks:spark-csv_2.10:1.5.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps6278997845129245926/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps6278997845129245926/https/repo1.maven.org/maven2/com/databricks/spark-csv_2.10/1.5.0/spark-csv_2.10-1.5.0.jar
-> New file at /tmp/toree_add_deps6278997845129245926/https

In [2]:
import com.evan.kaggle.se.FeatureEngineering._
val sqlContext = org.apache.spark.sql.SQLContext.getOrCreate(sc)
import sqlContext.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import scala.collection.immutable.HashSet

In [3]:
// val files = List("cooking")//, "crypto", "robotics", "biology", "travel", "diy")

// /* Load and print all files */
// val df_all = files.map(f => {
//                         sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("../dat/"+f+"_clean.csv")
//                    }).reduce(_ unionAll _).filter($"content" !== "")

Name: Syntax Error.
Message: 
StackTrace: 

In [4]:
val samp = sqlContext.read.parquet("union.parquet").sample(false, .5)

In [5]:
samp.count

43496

In [6]:
val feat_udf = udf((t: String, c: String, ta: String) => 
    makeTrFeatures(3)(t, c, ta))

In [7]:
val flat = samp.select(explode(feat_udf($"title", $"content", $"tags")))
val flat2 = flat.select($"col.*")

In [8]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer}
import org.apache.spark.ml.feature.VectorAssembler

//change name
val data = flat2.withColumn("isTag", when($"isTag", lit("Y")).otherwise(lit("N"))).
    withColumn("hasUpper", when($"hasUpper", lit(1.0)).otherwise(0.0))

val labelIndexer = new StringIndexer().
  setInputCol("isTag").
  setOutputCol("indexedLabel").
  fit(data)

val hashingTF = new HashingTF().setInputCol("posTags").
    setInputCol("depTags").
    setOutputCol("rawFeatures").setNumFeatures(20)

val idf = new IDF().setInputCol("rawFeatures").
    setOutputCol("text_features")

val assembler = new VectorAssembler().
    setInputCols(Array("text_features", "relPos", "hasUpper", "numWords")).
    setOutputCol("features")

In [9]:
// Split the data into training and test sets (30% held out for testing)
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

// Train a RandomForest model.
val rf = new RandomForestClassifier().setLabelCol("indexedLabel").
  setFeaturesCol("features").
  setNumTrees(10)

// Convert indexed labels back to original labels.
val labelConverter = new IndexToString().
  setInputCol("prediction").
  setOutputCol("predictedLabel").
  setLabels(labelIndexer.labels)

In [ ]:
// Chain indexers and forest in a Pipeline
val pipeline = new Pipeline().setStages(Array(labelIndexer, hashingTF, idf, assembler, rf, labelConverter))

// Train model.  This also runs the indexers.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

// Select example rows to display.
predictions.select("predictedLabel", "indexedLabel", "features").show(5)

// Select (prediction, true label) and compute test error
val evaluator = new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").
  setPredictionCol("prediction").
  setMetricName("precision")
val accuracy = evaluator.evaluate(predictions)
println("Test Error = " + (1.0 - accuracy))

In [ ]:
data.write.save("featurized.parquet")